In [1]:
 # Importing the libraries 
import cv2 
import numpy as np 
  
# Reading the image and converting the image to B/W 
image = cv2.imread('./assets/desert_high_res.png') 
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
gray_image = np.float32(gray_image) 
  
# Applying the function 
dst = cv2.cornerHarris(gray_image, blockSize=2, ksize=3, k=0.04) 
  
# dilate to mark the corners 
dst = cv2.dilate(dst, None) 
dst[dst > 0.1 * dst.max()] = 255
dst[dst <= 0.1 * dst.max()] = 0
# image[dst > 0.01 * dst.max()] = [0, 255, 0] 

plt.imshow(dst, cmap='gray')

NameError: name 'plt' is not defined

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

harris_window_size = 5

def my_harris(img_dir,window_size,k,threshold):

    img = cv2.imread(img_dir)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    img_gaussian = cv2.GaussianBlur(gray,(3,3),0)
    
    #   Step 1 - Calculate the x e y image derivatives (dx e dy)
    dx = cv2.Sobel(img_gaussian, cv2.CV_64F, 1, 0, ksize=3)
    dy = cv2.Sobel(img_gaussian, cv2.CV_64F, 0, 1, ksize=3)
    # dy, dx = np.gradient(gray)

    #   Step 2 - Calculate product and second derivatives (dx2, dy2 e dxy)
    Ix2=np.square(dx)
    Iy2=np.square(dy)
    Ixy=dx*dy

    offset = int( window_size / 2 )
    #   Step 3 - Calcular a soma dos produtos das derivadas para cada pixel (Sx2, Sy2 e Sxy)
    print ("Finding Corners...")
            
    harris_window = np.ones((harris_window_size, harris_window_size))
    Sx2 = convolve2d(Ix2, harris_window, mode='same')
    Sy2 = convolve2d(Iy2, harris_window, mode='same')
    Sxy = convolve2d(Ixy, harris_window, mode='same')

    # Calculate the determinant and the trace of the structure tensor
    det = Sx2 * Sy2 - (Sxy ** 2)
    trace = Sx2 + Sy2

    # Calculate the response of the detector
    matrix_R = det - k * (trace ** 2)
    
    #   Step 6 - Apply a threshold
    cv2.normalize(matrix_R, matrix_R, 0, 1, cv2.NORM_MINMAX)
    corners = np.zeros_like(matrix_R)
    corners[matrix_R > threshold] = 1
    return corners
                


my_harris('./assets/desert_high_res.png', 5, 0.04, 0.30) # Change this path to one that will lead to your image

Finding Corners...


NameError: name 'harris_window_size' is not defined